In [1]:
!pip install janome japanize_matplotlib

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from tensorflow.keras.utils import get_file

gz_file_path = get_file('examples_pd.gz', 'ftp://ftp.monash.edu/pub/nihongo/examples_pd.gz',)

In [3]:
import gzip
import pandas as pd

with open(gz_file_path, 'rb') as fd:
    gzip_fd = gzip.GzipFile(fileobj=fd)
    raw_lines = gzip_fd.readlines()


In [4]:
import re

NUM_EXAMPLES = 30000

re_line = re.compile(r'(A|B):\s(.+)\t(.+)(?:#ID=\d+\r\n)')

en = []
ja = []

for raw_line in raw_lines[:NUM_EXAMPLES]:
    raw_line = raw_line.decode('euc_jisx0213')
    m = re_line.match(raw_line)
    if m and m[1] == 'A':
        ja.append(m[2])
        en.append(m[3])
print(ja[:5])
print(en[:5])

['＆という記号は、ａｎｄを指す。', '＆のマークはａｎｄの文字を表す。', '（自転車に乗って）フーッ、この坂道はきついよ。でも帰りは楽だよね。', '実のところ物価は毎週上昇している。', '〜と痛切に感じている。']
["The sign '&' stands for 'and'.", 'The mark "&" stands for "and".', '(On a bicycle) Whew! This is a tough hill. But coming back sure will be a breeze.', 'As it is, prices are going up every week.', 'I was acutely aware that..']


In [5]:
import unicodedata

def normalize_en(s):
    return unicodedata.normalize('NFD', s)

def normalize_ja(s):
    return unicodedata.normalize('NFKC', s)

In [6]:
from janome.tokenizer import Tokenizer

t_wakati = Tokenizer(wakati=True)

def tokenize_japanese(text):
    return ' '.join(list(t_wakati.tokenize(text)))

In [7]:
import re

def preprocess_en(w):
    w = normalize_en(w.lower().strip())

    # 単語とそのあとの句読点の間にスペースを挿入
    # 例：　"he is a boy." => "he is a boy ."
    # 参照：- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    w = w.rstrip().strip()

    # 文の開始と終了のトークンを付加
    # モデルが予測をいつ開始し、いつ終了すれば良いかを知らせるため
    w = '<start> ' + w + ' <end>'
    return w

def preprocess_ja(w):
    w = normalize_ja(w)

    w = tokenize_japanese(w)
    w = re.sub(r'[" "]+', " ", w)
    w = w.rstrip().strip()

    # 文の開始と終了のトークンを付加
    # モデルが予測をいつ開始し、いつ終了すれば良いかを知らせるため
    w = '<start> ' + w + ' <end>'
    return w


In [8]:
en_example = "The sign '&' stands for 'and'."
ja_example = '＆という記号は、ａｎｄを指す。'
print(preprocess_en(en_example))
print(preprocess_ja(ja_example))

<start> the sign '&' stands for 'and' . <end>
<start> & という 記号 は 、 and を 指す 。 <end>


In [9]:
en = [preprocess_en(s) for s in en]
ja = [preprocess_ja(s) for s in ja]
print(en[-1])
print(ja[-1])

<start> this toothbrush is not used by my mother . <end>
<start> この 歯ブラシ を 使っ て いる の は 母 で は ない 。 <end>


In [10]:
import tensorflow as tf

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post')
    return tensor, lang_tokenizer

In [11]:
def create_dataset(targ_lang, inp_lang):
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [12]:
input_tensor, target_tensor, inp_lang, targ_lang = create_dataset(ja, en)

In [13]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [14]:
# ターゲットテンソルの最大長を計算
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [15]:
from sklearn.model_selection import train_test_split

# 80-20で分割を行い、訓練用と検証用のデータセットを作成
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 長さを表示
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

12000 12000 3000 3000


In [16]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [17]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
710 ----> we're
42 ----> all
3462 ----> booked
54 ----> up
26 ----> at
918 ----> 6
1283 ----> p
3 ----> .
1160 ----> m
9 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
160 ----> 6
56 ----> 時
4 ----> は
4168 ----> 満席
17 ----> です
3 ----> 。
2 ----> <end>


In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [19]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 47]), TensorShape([64, 66]))

In [20]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x):
        x = self.embedding(x)
        output, *states = self.lstm(x)
        return output, states


In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units)

# サンプル入力
sample_output, sample_hidden = encoder(example_input_batch)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))
print ('Encoder Carry state shape: (batch size, units) {}'.format(sample_hidden[1].shape))

In [22]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # 埋め込み層を通過したあとの x の shape  == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # Embeddingの出力と、エンコーダ出力を GRU 層に渡す
        output, *states = self.lstm(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, states

In [25]:
decoder = Decoder(vocab_tar_size, embedding_dim, units)

sample_decoder_output, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 10804)


In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
                             from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [27]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [28]:
@tf.function
def train_step(inp, targ):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher Forcing - 正解値を次の入力として供給
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)

            loss += loss_function(targ[:, t], predictions)

            # Teacher Forcing を使用
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [29]:
import time

EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ)
        total_loss += batch_loss

        if batch % 100 == 0:
              print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
    # 2 エポックごとにモデル（のチェックポイント）を保存
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 1.7436
Epoch 1 Batch 100 Loss 0.9303
Epoch 1 Loss 1.0137
Time taken for 1 epoch 66.8834240436554 sec

Epoch 2 Batch 0 Loss 0.8175
Epoch 2 Batch 100 Loss 0.8236
Epoch 2 Loss 0.7944
Time taken for 1 epoch 27.65331506729126 sec

Epoch 3 Batch 0 Loss 0.7370
Epoch 3 Batch 100 Loss 0.7876
Epoch 3 Loss 0.7177
Time taken for 1 epoch 27.518179893493652 sec

Epoch 4 Batch 0 Loss 0.6718
Epoch 4 Batch 100 Loss 0.6098
Epoch 4 Loss 0.6618
Time taken for 1 epoch 27.859076499938965 sec

Epoch 5 Batch 0 Loss 0.5678
Epoch 5 Batch 100 Loss 0.6280
Epoch 5 Loss 0.6140
Time taken for 1 epoch 27.60209846496582 sec

Epoch 6 Batch 0 Loss 0.5603
Epoch 6 Batch 100 Loss 0.6357
Epoch 6 Loss 0.5703
Time taken for 1 epoch 27.81960368156433 sec

Epoch 7 Batch 0 Loss 0.5467
Epoch 7 Batch 100 Loss 0.5660
Epoch 7 Loss 0.5293
Time taken for 1 epoch 27.983044862747192 sec

Epoch 8 Batch 0 Loss 0.4494
Epoch 8 Batch 100 Loss 0.4978
Epoch 8 Loss 0.4941
Time taken for 1 epoch 28.01467275619507 sec

Epoch 

In [30]:
import numpy as np

def evaluate(sentence):
    sentence = preprocess_en(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    enc_out, enc_hidden = encoder(inputs)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input,
                                          dec_hidden)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # 予測された ID がモデルに戻される
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [31]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))


In [32]:
# checkpoint_dir の中の最後のチェックポイントを復元
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [33]:
translate('It is necessary that the bill pass the diet.')

Input: <start> it is necessary that the bill pass the diet . <end>
Predicted translation: あの 人 に は とかく の うわさ が あっ た 。 <end> 


In [34]:
def predict(inputs):
    inputs = tf.convert_to_tensor([inputs])
    predicted_seq = []
    
    enc_out, enc_hidden = encoder(inputs)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input,
                                          dec_hidden)

        predicted_id = tf.argmax(predictions[0]).numpy()

        predicted_seq.append(predicted_id)

        if targ_lang.index_word[predicted_id] == '<end>':
            return predicted_seq

        # 予測された ID がモデルに戻される
        dec_input = tf.expand_dims([predicted_id], 0)

    return predicted_seq
    

In [39]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

bleu_scores = []
for i, input_en in enumerate(input_tensor_val):
    predicted_ja = predict(input_en)
    tokens_input = [inp_lang.index_word[id] for id in input_en if id > 2]
    tokens_target = [targ_lang.index_word[id] for id in target_tensor_val[i] if id > 2]
    tokens_predicted = [targ_lang.index_word[id] for id in predicted_ja if id > 2]
    bleu_score = sentence_bleu(tokens_target, tokens_predicted, smoothing_function=SmoothingFunction().method1)
    bleu_scores.append(bleu_score)
    print('Source:{}'.format(' '.join(tokens_input)))
    print('Target:{}'.format(''.join(tokens_target)))
    print('Prediction:{}'.format(''.join(tokens_predicted)))
    print('BLEU score:', bleu_score)
    print()
    
avg_bleu = np.average(bleu_scores)
print('Average BLEU:', avg_bleu)

Source:i'd like you to pick up the pace a little on this job .
Target:この作業、もうちょっとスピードアップできないかなあ。
Prediction:あなたが私達を手伝ってくれたらどうかのか。
BLEU score: 0.018798317647335087

Source:may i have your name and room number , please ?
Target:お名前と部屋番号をお願いします。
Prediction:あなたの名前を教えてください。
BLEU score: 0.03303164318013808

Source:i'd like to ask a favor of you .
Target:お願いしたい事があるのですが。
Prediction:あなたが私達を手伝ってくれたらどうしようか。
BLEU score: 0.0169861974906263

Source:japanese management must learn how to deal with american workers , he said .
Target:「日本の経営陣は米国の労働者の扱い方を身につけなければいけない」と彼は言った。
Prediction:あの人たちが休暇で留守にしている間、隣の人たちが犬の面倒をみてやった。
BLEU score: 0.010438074937942448

Source:the ink stain will not wash out .
Target:インクの染みが洗濯しても落ちない。
Prediction:あの人は眼の横にほくろがある。
BLEU score: 0.024808415001701823

Source:in 1972 , dr . francine patterson started to teach sign language to koko .
Target:1972年に、フランシーヌ・パターソン博士はココに手話を教え始めた。
Prediction:「私の幹ももうありません」と木は言いました。
BLEU score: 0.015138514598766055

Source:when will you get ready to lea